# Clustering

Using clustering, we may be able to find particular clusters of patients that have a higher amount of Alzheimer's disease than others. Then, based on the differences between clusters, we might be able to find a pattern.

In [ ]:
import data_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.decomposition import PCA


def dissimilarity_score(v1, v2):
    # For categorical variables, calculating the euclidian distance is not
    # correct. Therefore, the amount of differences should just be counted.
    return np.sum(v1 != v2)


def euclidian_distance(v1, v2):
    return np.sqrt(np.sum((v1 - v2)**2))


def assign_cluster(prototypes, clusters, point):
    dissimilarities = []
    for pt in prototypes:
        v_n = num_matrix[point]
        v_c = cat_matrix[point]
        pt_v_n = pt[0]
        pt_v_c = pt[1]
        dissimilarity = dissimilarity_score(
            pt_v_n, v_n) + euclidian_distance(pt_v_c, v_c)
        dissimilarities.append(dissimilarity)

    dissimilarities = np.array(dissimilarities)
    cluster = np.where(dissimilarities == dissimilarities.min())[0][0]
    clusters[cluster].append((num_matrix[point], cat_matrix[point]))

    return clusters


def calc_prototype(cluster, init_index):
    if len(cluster) == 0:
        # Whenever a cluster is empty, reset the prototype to what the prototype
        # originally was for this cluster. Taking the means/modes of all variables
        # or setting it to 0 did not work well, as it would result in lots of
        # empty clusters.
        return (num_matrix[init_index], cat_matrix[init_index])
    num_cluster = np.vstack([point[0] for point in cluster])
    num_prototype = np.mean(num_cluster, axis=0)
    cat_cluster = np.vstack([point[1] for point in cluster])
    cat_prototype = mode(cat_cluster, axis=0).mode

    return (num_prototype, cat_prototype)


def k_prototypes_clustering(num_matrix, cat_matrix, k, verbose=False, initial_prototypes=None):
    if initial_prototypes is None:
        # Randomly select k initial prototype indexes
        prototypes_indices = np.random.randint(0, len(num_matrix), k)
        init_indexes = prototypes_indices.copy()
        if verbose:
            print(f'Selected initial prototypes indices: {prototypes_indices}')

        # Initialize prototypes as a list of tuples with numeric and categorical data
        prototypes = [(num_matrix[i], cat_matrix[i])
                      for i in prototypes_indices]
    else:
        init_indexes = initial_prototypes.copy()
        if len(initial_prototypes) != k:
            raise ValueError("The number of initial prototypes must match k.")
        prototypes = [(num_matrix[i], cat_matrix[i])
                      for i in initial_prototypes]

    clusters = [[] for _ in range(k)]

    while True:
        for i in range(len(num_matrix)):
            clusters = assign_cluster(prototypes, clusters, i)

        new_prototypes = [calc_prototype(clusters[i], init_indexes[i]) for i in range(len(clusters))]

        # Check for convergence, meaning that the prototypes did not change
        # during the current iteration, indicating that the algorithm is done.
        done = True
        for i in range(len(prototypes)):
            if not np.array_equal(prototypes[i][0], new_prototypes[i][0]) or \
               not np.array_equal(prototypes[i][1], new_prototypes[i][1]):
                done = False
                break

        if done:
            break

        prototypes = new_prototypes
        clusters = [[] for _ in range(k)]

    if verbose:
        print(f'--- {k} clusters found ---')
        for i, cluster in enumerate(clusters):
            print(f' - Cluster {i + 1} with size={len(cluster)}')

    return clusters, prototypes, prototypes_indices


data = data_reader.get_data_dict('./data/alzheimers_disease_data.csv')

# Numerical columns
num_cols = ['BMI', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL',
            'CholesterolHDL', 'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment',
            'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality',
            'ADL']
# Categorical columns
cat_cols = ['FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes',
            'Depression', 'Hypertension', 'MemoryComplaints', 'BehavioralProblems',
            'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
            'Forgetfulness', 'HeadInjury', 'Smoking', 'Ethnicity', 'Gender',
            'EducationLevel', 'Diagnosis']

num_matrix = np.vstack(tuple(data[col] for col in num_cols)).T
cat_matrix = np.vstack(tuple(data[col] for col in cat_cols)).T

clusters, prototypes, _ = k_prototypes_clustering(num_matrix, cat_matrix, 5, True)

Now we can try to find the best starting prototypes, based on which of them creates clusters with the least total distance between the points and their cluster centroids.

In [ ]:
num_runs = 50
k = 4

best_clusters = None
best_prototypes = None
best_score = float('inf')
best_prototype_index = None

# Perform multiple runs to find which initial prototypes produce the smallest
# total difference, which is then marked as the best.
for run in range(num_runs):
    clusters = [[] for _ in range(k)]

    clusters, prototypes, prototype_index = k_prototypes_clustering(num_matrix, cat_matrix, k)

    total_distance = 0
    for cluster_idx, cluster in enumerate(clusters):
        for point in cluster:
            num_distance = euclidian_distance(point[0], prototypes[cluster_idx][0])
            cat_distance = dissimilarity_score(point[1], prototypes[cluster_idx][1])
            total_distance += num_distance + cat_distance

    if total_distance < best_score:
        best_score = total_distance
        best_clusters = clusters
        best_prototypes = prototypes
        best_prototype_index = prototype_index

print(f"Best clustering result found after {num_runs} runs:")
print(f"Within-cluster distance: {best_score:.2f}")
for i, cluster in enumerate(best_clusters):
    print(f"Cluster {i + 1}: {len(cluster)} points")


In [ ]:
print(best_prototype_index)

In [ ]:
pca = PCA(n_components=2)
cluster_data = [point[0] for cluster in best_clusters for point in cluster]
cluster_labels = [i for i, cluster in enumerate(best_clusters) for _ in cluster]

reduced_data = pca.fit_transform(cluster_data)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=cluster_labels, cmap='tab10')
plt.title("Cluster Visualization")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster")
plt.show()


In [ ]:
# Reproduce results by using the found best indexes.
prototypes = best_prototype_index
prototypes = [(num_matrix[i], cat_matrix[i]) for i in prototypes]

clusters, prototypes, prototype_index = k_prototypes_clustering(num_matrix, cat_matrix, k)

print("Reproduced clustering result:")
for i, cluster in enumerate(clusters):
    print(f"Cluster {i + 1}: {len(cluster)} points")


In [ ]:
pca = PCA(n_components=2)
cluster_data = [point[0] for cluster in clusters for point in cluster]
cluster_labels = [i for i, cluster in enumerate(clusters) for _ in cluster]

reduced_data = pca.fit_transform(cluster_data)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=cluster_labels)
plt.title("Cluster Visualization")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(label="Cluster")
plt.show()

Neither of the clustering visualisations show anything that resembles clusters, which can be explained by the fact that the clusters are made up of many more components than just these two. To find if there are any differences, we can try to use the mean and modes of these different components.

In [ ]:
means = []
modes = []
ratios = []
for cluster in clusters:
    cluster_num_matrix = np.array([item[0] for item in cluster])
    cluster_cat_matrix = np.array([item[1] for item in cluster])
    diagnosis = cluster_cat_matrix[:, -1]
    count_zeros = np.sum(diagnosis == 0)
    count_ones = np.sum(diagnosis == 1)
    ratios.append(count_ones/(count_zeros + count_ones))
    cluster_means = np.mean(cluster_num_matrix, axis=0)
    cluster_modes = mode(cluster_cat_matrix, axis=0).mode
    means.append(cluster_means)
    modes.append(cluster_modes)


print('-- Ratios of diagnosis --')
print(ratios)

print('-- Numerical Columns means --')
for i in range(len(num_cols)):
    print(num_cols[i])
    print([mean[i] for mean in means])

print('-- Categorical Columns modes --')
for i in range(len(cat_cols)):
    print(cat_cols[i])
    print([mode[i] for mode in modes])


As we can see there are no significant differences between the clusters. Sometimes, the clustering algorithm returns clusters where the diagnoses are spread evenly (about 33% of patients have one per cluster). Even when the algorithm returns clusters where there are some with (close to) 0 diagnoses and others where (close to) everyone has a diagnosis, we still cannot find any differences.